[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Wajeed-msft/open-ai-demo/blob/main/OpenAI-Demo.ipynb)

# Open AI - try outs

We use this notebook to create embeddings with OpenAI and use completion model to get responses. Required installs for this notebook are:

In [54]:
!pip install -qU openai datasets tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 50.0 MB/s eta 0:00:00


### Embedding with OpenAI

We begin by initializing the embedding model. For this we need [OpenAI API keys](https://beta.openai.com/signup).

In [ ]:
import openai

# TODO do we need org key?
#openai.organization = "OPENAI_ORG_KEY"
# get this from top-right dropdown on OpenAI under organization > settings
openai.api_key = "OPEN_API_KEY"
# get API key from top-right dropdown on OpenAI website

openai.Engine.list()  # check we have authenticated

The `openai.Engine.list()` function should return a list of models that we can use. One of those is `text-davinci-003` that we will use for creating completion like so:

In [ ]:
res = openai.Completion.create(
    engine="text-davinci-003",
    prompt="""Write a tagline for an ice cream shop."""   
)
res

In [ ]:
res['choices'][0]["text"]

## Let's try Classification
To create a text classifier with the API, we provide a description of the task and a few examples. In this example, we show how to classify the sentiment of Tweets.

In [ ]:
res = openai.Completion.create(
    engine="text-davinci-003",
    max_tokens=100,
    prompt="""Decide whether a Tweet's sentiment is positive, neutral, or negative.
            Tweet: I loved the new Batman movie!
            Sentiment:"""   
)
print(res['choices'][0]["text"])

## Let's generate [Conversation](https://platform.openai.com/docs/guides/completion/conversation) with AI

In [ ]:
res = openai.Completion.create(
    engine="text-davinci-003",
    max_tokens=100,
    prompt="""The following is a conversation with an AI assistant. The assistant is helpful, creative, clever, and very friendly.
            Human: Hello, who are you?
            AI: I am an AI created by OpenAI. How can I help you today?
            Human:""")

print(res['choices'][0]["text"])


## Let's ask AI to [Generate](https://platform.openai.com/docs/guides/completion/generation) ideas

In [ ]:
res = openai.Completion.create(
    engine="text-davinci-003",
    max_tokens=200,
    prompt="""Brainstorm some ideas combining VR and fitness.:"""   
        )

print(res['choices'][0]["text"])

In [ ]:
res = openai.Completion.create(
    engine="text-davinci-003",
    max_tokens=200,
    prompt="""GPT-3 is a very nice AI
              That's pretty good at writing replies
              When it's asked a question
              It gives its suggestion
              This is a poem it made that rhymes
              Make this in the voice of GPT-3:"""   
        )

print(res['choices'][0]["text"])

## Let's generate images

In [ ]:
res = openai.Image.create(
  prompt="A cute baby cat",
  n=1,
  size="256x256"
)
res

In [ ]:
res = openai.Image.create(
  prompt="A cute baby cat playing with a dog",
  n=1,
  size="256x256"
)
res

## Question Answering using Embeddings

source: https://github.com/openai/openai-cookbook/blob/main/examples/Question_answering_using_embeddings.ipynb

In [55]:
import numpy as np
import openai
import pandas as pd
import pickle
import tiktoken

COMPLETIONS_MODEL = "text-davinci-003"
EMBEDDING_MODEL = "text-embedding-ada-002"

In [59]:
prompt = "Who won the 2020 Summer Olympics men's high jump?"

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

"Marcelo Chierighini of Brazil won the gold medal in the men's high jump at the 2020 Summer Olympics."

### Marcelo is a gold medalist swimmer, and, we assume, not much of a high jumper! Evidently GPT-3 needs some assistance here.

In [60]:
prompt = """Answer the question as truthfully as possible, and if you're unsure of the answer, say "Sorry, I don't know".

Q: Who won the 2020 Summer Olympics men's high jump?
A:"""

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

"Sorry, I don't know."

In [ ]:
prompt = """Answer the question as truthfully as possible using the provided text, and if the answer is not contained within the text below, say "I don't know"

Context:
The men's high jump event at the 2020 Summer Olympics took place between 30 July and 1 August 2021 at the Olympic Stadium.
33 athletes from 24 nations competed; the total possible number depended on how many nations would use universality places 
to enter athletes in addition to the 32 qualifying through mark or ranking (no universality places were used in 2021).
Italian athlete Gianmarco Tamberi along with Qatari athlete Mutaz Essa Barshim emerged as joint winners of the event following
a tie between both of them as they cleared 2.37m. Both Tamberi and Barshim agreed to share the gold medal in a rare instance
where the athletes of different nations had agreed to share the same medal in the history of Olympics. 
Barshim in particular was heard to ask a competition official "Can we have two golds?" in response to being offered a 
'jump off'. Maksim Nedasekau of Belarus took bronze. The medals were the first ever in the men's high jump for Italy and 
Belarus, the first gold in the men's high jump for Italy and Qatar, and the third consecutive medal in the men's high jump
for Qatar (all by Barshim). Barshim became only the second man to earn three medals in high jump, joining Patrik Sjöberg
of Sweden (1984 to 1992).

Q: Who won the 2020 Summer Olympics men's high jump?
A:"""

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")